# Julia Extra! The Standard Map & Time-Reversibility.

For a quick introduction to Julia, check out this series of Julia notebooks. Hopefully, you'll see most things are similar to python, but there are important structural and semantic differences that I think you can tell by reading more Julia code. In this first notebook, we'll be looking at some cool dynamical systems. More specifically, we'll be looking at concepts of irreversibility in the Standard Map aka the Chirikov Map.

We're going to be going through an analysis of the classical Chirikov Map done in this paper, but know that there is a quantum formulation of this problem.

The motivation behind studying a system like the Standard map was the formulation of quantum statistical effects as stochasticity on a classical level. Shepelyanksy calls these classically stochastic quantum systems (SQS).In his paper, he tries to demonstrate that the dynamics of SQS are close to classically stochastic. 



We begin by considering the following Hamiltonian:
$$ H = -\frac{\hbar^2}{2J} \frac{\partial^2}{\partial^2 \theta} + \tilde{k} \cos{(\theta)} \delta_{\tilde{T}}(\tau)$$

This Hamiltonian describes the motion of a rotator in an external field (compare to a particle on a sphere: $H = L^2/2J$, J is the moment of inertia). For the purpose of this notebook and the paper, we consider $J=1$. $\tilde{k}$ is the magnitude of the perturbation, $\delta_{\tilde{T}}$ is a periodic delta function that "kicks" the rotator, adding energy every now and then.

We can consider the system as a discrete mapping, where time is defined by the kick #/the discrete time units are the kick events, $p_i, \theta_i \in \R$
$$ p_{n + 1} = p_{n} + \tilde{k}\sin{(\theta)} $$
$$ \theta_{n + 1} = \theta_{n} + p_{n + 1} \pmod{2\pi}$$

We're going to make a change of variables now:
$\theta = 2\pi x$, $\tilde{k} = k/(2\pi)$, $y = p$. Now, it makes sense to consider $x \mod{1}$. If we want to define this mapping on a torus, we can make both $x, y \pmod{1}$. 

Now, we instantiate two definitions of the standard map: the first one is to visualize what it looks like on a torus, and the second one, we will use to investigate time reversibility.

In [ ]:
# These type annotations don't really do anything, they're more for you than functionality.
function standard_map(x::Real, y::Real, k::Float64)
    y_new = mod(y + (k / (2 * pi)) * sin(2 * pi * x), 1) # note pi. pi is of type Irrational, and is kept as such until the end.
    x_new = mod(x + y_new, 1)                            # when a calculation is done. mod(x, 1) is the same as x % 1 in python
    return x_new, y_new
end;

In [ ]:
function standard_map_cylinder(x::Real, y::Real, k::Float64)
    y_new = y + (k / (2 * pi)) * sin(2 * pi * x)
    x_new = mod(x + y_new, 1)
    return x_new, y_new
end;

Here are some of the packages we are going to need.

In [ ]:
using Plots, LaTeXStrings

The following cell takes a *long* time to run. It took about 45 minutes on my laptop-- you can trade off accuracy for different values in each of the loop, but the goal here is to get a view of the phase space. The standard map is one of the most visually striking dynamical systems, exhibits a plethora of interesting dynamical behaviors. The value of $k$ I'm plotting at is the critical value, but feel free to change it and play around with it. Imagine setting $k$ to zero: then what you would expect is just lines in the plane. It's interesting to see how this system changes by changing the strength of the perturbation $\tilde{k}$. Remember that it's the $\tilde{k}$ term that makes it an open system, adding energy by periodically forcing the rotator, so this might have some insight into a continuous parameter between a closed (particle on a ring with constant energy) and an open system.

In [ ]:
# Simulating the Standard Map also known as the Chirikov Standard Map at a particular parameter value.
# Try changing the value of k to see how the dynamics change.
# The original value was k = 0.971635
# This takes a long time to run. Alternatively, you could just look up what the standard map on a torus looks like
k = 0.971635
P = plot()
for i in range(1, 100, step = 1)
    x = rand()
    y = rand()
    for j in range(1, 500, step = 1)
        x, y = standard_map(x, y, k)
        scatter!(P, [x], [y], color = :black, legend = false, ms=1)
    end
end
P



Okay, now that we have a cool picture that highlights the intricacy of the standard map, let's turn back to the standard map with $y$ free and $x \mod{1}$. We can look at some of the cool computational power in Julia by considering the time-reversal symmetry of the Hamiltonian. While it is true that $t \rightarrow -t$ leaves the Hamiltonian unchanged ($p^2 \rightarrow (-p)^2$ is translationally invariant for $x$), classically the Kolmogorov-Sinai Entropy (KS-Entropy) $h$ is nonzero $(h > 0)$. This suggests that small perturbations (on the order of computer error) cause two trajectories close in phase space to diverge (quickly). This makes the dynamcis of this system time-irreversible in practice. To demonstrate this, let's re-create figure 3 from the original paper.

In [ ]:
# Parameters 
k = 5.0     # standard map k-coefficient
averaging=10000 # the number of averaged trajectories (10^4) ~ 10 times as many as the original paper (<10 minutes)
forward_time = 100 # the number of forward iterations (100) before time-reversal
backward_time = 100 # the number of backward iterations (100) after time-reversal
data = []

for j in range(1, averaging)
    x0 = rand()
    y0 = rand()

    x = [x0]
    y = [y0]
    x2 = [x0]
    y2 = [y0]

    for i in range(1, forward_time)
        x0, y0 = standard_map2(x0, y0, k)
        push!(x, x0)
        push!(y, y0)
        push!(x2, x0)
        push!(y2, y0)
    end

    x02 = x0
    y02 = y0

    # Time reversal
    for i in range(1, backward_time)
        x02, y02 = standard_map2(x02, -y02, k)
        x0, y0 = standard_map2(x0, y0, k)
        push!(x2, x02)
        push!(y2, y02)
        push!(x, x0)
        push!(y, y0)
    end
    
    push!(data, [x, y, x2, y2])
end

In [ ]:
# if we want to calculate the energy of the standard map, we should remember that it comes from a periodically kicked oscillator.
# this is an open system-- energy has to be added to periodically kick the oscillator. So let's plot a graph of energy
# versus time for the standard map.
# this took ~7 minutes to run on my laptop
# remember that x ~ x, y ~ p 
# E = <p(t)^2>/2
# we need to take the average of p at every time point
p_arr = [data[i][2] for i in range(1, length(data))] # array of p-arrays
psqr = [p_arr[i] .^ 2 for i in range(1, length(p_arr))] # array of p^2 arrays
avg_p2 = [sum(psqr)/length(p_arr[i]) for i in range(1, length(p_arr))] # array of average p's
E = [avg_p2[i] / 2 for i in range(1, length(p_arr))] # array of energies
print(E)
 

# now we can plot the energy versus time

#plot!(t./k, color = :black, legend = false)

In [ ]:
p_arr2 = [data[i][4] for i in range(1, length(data))] # array of p-arrays
psqr2 = [p_arr2[i] .^ 2 for i in range(1, length(p_arr2))] # array of p^2 arrays
avgp2 = [sum(psqr2)/length(p_arr2[i]) for i in range(1, length(p_arr2))] # array of average p's
E2 = [avgp2[i] / 2 for i in range(1, length(p_arr2))]
t2 = range(0, 200)

In [ ]:
# this took ~ 2 minutes to run on my laptop. 
plot(4 .* E ./k^2, xlabel = L"t", ylabel = L"\frac{4E}{k^2}", legend = false)
plot!(t2, 4 .* E2 ./k^2, xlabel = L"t", ylabel = L"\frac{4E}{k^2}", legend = false, color = :green, show=true)

# To Do:
Implement quantum standard map and show that time reversal is preserved in energy

# References: 
- [Some statistical properties of simple classically stochastic quantum systems](https://www.sciencedirect.com/science/article/pii/0167278983903184?fr=RR-2&ref=pdf_download&rr=831965075fea0bef)
- [A universal instability of many-dimensional oscillator systems](https://www.sciencedirect.com/science/article/pii/0370157379900231)
- [Chirikov standard map:Scholarpedia](http://www.scholarpedia.org/article/Chirikov_standard_map#1)
